##### Imports

In [1]:
import pandas as pd
import nltk
import numpy as np
from gensim.models import Word2Vec

##### Load in Data

In [3]:
text_df = pd.read_pickle("merged_training.pkl")
text_df = text_df.reset_index(drop = True)
text_df

,text,emotions
0,i feel awful about it too because it s my job ...,sadness
1,im alone i feel awful,sadness
2,ive probably mentioned this before but i reall...,joy
3,i was feeling a little low few days back,sadness
4,i beleive that i am much more sensitive to oth...,love
...,...,...
416804,that was what i felt when i was finally accept...,joy
416805,i take every day as it comes i m just focussin...,fear
416806,i just suddenly feel that everything was fake,sadness
416807,im feeling more eager than ever to claw back w...,joy


##### Tokenize Text

In [4]:
# store data by words
data_by_words = []
# loop through texts
for i in text_df['text']:
    # get words, tokenize
    value = nltk.word_tokenize(i)
    data_by_words.append(value)
data_by_words

[['i',
  'feel',
  'awful',
  'about',
  'it',
  'too',
  'because',
  'it',
  's',
  'my',
  'job',
  'to',
  'get',
  'him',
  'in',
  'a',
  'position',
  'to',
  'succeed',
  'and',
  'it',
  'just',
  'didn',
  't',
  'happen',
  'here'],
 ['im', 'alone', 'i', 'feel', 'awful'],
 ['ive',
  'probably',
  'mentioned',
  'this',
  'before',
  'but',
  'i',
  'really',
  'do',
  'feel',
  'proud',
  'of',
  'myself',
  'for',
  'actually',
  'keeping',
  'up',
  'with',
  'my',
  'new',
  'years',
  'resolution',
  'of',
  'monthly',
  'and',
  'weekly',
  'goals'],
 ['i', 'was', 'feeling', 'a', 'little', 'low', 'few', 'days', 'back'],
 ['i',
  'beleive',
  'that',
  'i',
  'am',
  'much',
  'more',
  'sensitive',
  'to',
  'other',
  'peoples',
  'feelings',
  'and',
  'tend',
  'to',
  'be',
  'more',
  'compassionate'],
 ['i',
  'find',
  'myself',
  'frustrated',
  'with',
  'christians',
  'because',
  'i',
  'feel',
  'that',
  'there',
  'is',
  'constantly',
  'a',
  'talk',
  

##### Get Document Embeddings

In [5]:
model = Word2Vec(sentences = data_by_words, vector_size = 100)
model.train(data_by_words, total_examples = len(data_by_words), epochs = 50)

(271007412, 400544400)

In [6]:
model.save("word2vec.model")

In [7]:
embeddings_word_data = []
# for each document
for i in data_by_words:
    text = []
    # for each word in doc
    for j in i:
        # if word in model vocab get the embedding
        if j in model.wv.key_to_index:
            text.append(model.wv[j])
    # average embeddings across all word embeddings
    text = np.mean(text, axis = 0)
    embeddings_word_data.append(text)
embeddings_word_data = np.array(embeddings_word_data)
print(embeddings_word_data.shape)

c:\Users\jenni\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jenni\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(416809,)


<ipython-input-7-346ecf699279>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings_word_data = np.array(embeddings_word_data)


In [8]:
# save
np.save('cbow_embeddings.npy', embeddings_word_data)